In [ ]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from pathlib import Path
import statistics as stats
import argparse
from torch.utils.data import Sampler
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.modules import Module
# Set device
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from sklearn.metrics import accuracy_score, classification_report

# Dataset loading
transform = transforms.Compose([
    transforms.Resize((224, 224)),   
    transforms.ToTensor(),
])

In [ ]:
import torch
import torch.nn.functional as F

def class_cluster_loss(prototypes, features, labels, c_values, margin=1.0 ):
    """
    Compute the Class Cluster Loss for a batch.
    
    :param prototypes: Tensor of shape (num_classes, feature_dim) containing class prototypes
    :param features: Tensor of shape (batch_size, feature_dim) containing features of batch samples
    :param labels: LongTensor of shape (batch_size,) containing labels of batch samples
    :param margin: Float, the margin parameter for the loss calculation
    :param c: Float, the central distance parameter for intra-class compactness
    :return: Class Cluster Loss value for the batch
    """
    n_classes = prototypes.shape[0]
    loss = 0.0
    
    for i in range(n_classes):
        # Selecting the current class prototype and corresponding features and labels
        current_prototype = prototypes[i]
        class_mask = labels == i
        class_features = features[class_mask]
        #print("Prototype shape:", current_prototype.shape)
        #print("Class features shape:", class_features.shape)

        
        # Calculating distances from the current class features to its prototype
        pos_distances = torch.norm(class_features - current_prototype, p=2, dim=1)
        # Calculate c as the mean of these distances
        # Calculate c as the mean of these distances
        c = pos_distances.mean()
        #print('c', c)
        # Distance to the farthest positive example
        D_pro_max_p = torch.max(pos_distances)
        
        # Calculating distances to the prototype from features of other classes (negative examples)
        neg_mask = ~class_mask
        neg_features = features[neg_mask]
        neg_distances = torch.norm(neg_features - current_prototype, p=2, dim=1)
        
        # Distance to the closest negative example
        if len(neg_distances) > 0:
            D_pro_min_n = torch.min(neg_distances)
        else:
            # Handling cases where a class might have all the samples in the batch
            D_pro_min_n = torch.tensor(0.0).to(features.device)
        
        # Computing the first term of the CCL
        loss += F.relu(D_pro_max_p - D_pro_min_n + margin)
        # Ensure you're using the correct 'c' for the current class
        current_c = c
        # Additional term to ensure the farthest positive is not too far
        loss += F.relu(D_pro_max_p - current_c)
    
    # Averaging the loss over the number of classes
    loss /= n_classes
    
    return loss
import torch

def calculate_central_distance(prototypes, features, labels):
    num_classes = prototypes.shape[0]
    c_values = torch.zeros(num_classes, device=prototypes.device)
    
    for i in range(num_classes):
        class_mask = labels == i
        #print(class_mask)
        class_features = features[class_mask]
        
        if class_features.nelement() == 0:  # Check if there are no features for the current class
            c_values[i] = torch.tensor(0.0, device=prototypes.device)  # Assign nan or another placeholder
            continue  # Skip the rest of the loop for this class
        
        distances = torch.norm(class_features - prototypes[i], dim=1, p=2)
        c_values[i] = distances.mean()  # This will not be nan since class_features is not empty

    return c_values

margin = 0.00005
n_classes = 4

class PrototypicalLoss(Module):
    '''
    Loss class deriving from Module for the prototypical loss function defined below
    '''
    def __init__(self, n_support):
        super(PrototypicalLoss, self).__init__()
        self.n_support = n_support

    def forward(self, input, target):
        return prototypical_loss(input, target)#, self.n_support)

def prototypical_loss(input, target):#, n_support):
    '''
    Inspired by https://github.com/jakesnell/prototypical-networks/blob/master/protonets/models/few_shot.py

    Compute the barycentres by averaging the features of n_support
    samples for each class in target, computes then the distances from each
    samples' features to each one of the barycentres, computes the
    log_probability for each n_query samples for each one of the current
    classes, of appartaining to a class c, loss and accuracy are then computed
    and returned
    Args:
    - input: the model output for a batch of samples
    - target: ground truth for the above batch of samples
    - n_support: number of samples to keep in account when computing
      barycentres, for each one of the current classes
    '''
    target_cpu = target.to(device)
    input_cpu = input.to(device)
    features=input_cpu
    support_features, support_labels = features[:n_support], labels[:n_support]
    query_features, query_labels = features[n_support:], labels[n_support:]
        
    def supp_idxs(c):
        # FIXME when torch will support where as np
        return target_cpu.eq(c).nonzero()[:n_support].squeeze(1)

    # FIXME when torch.unique will be available on cuda too
    classes = torch.unique(target_cpu)
    n_classes = len(classes)
    #print('n_classes:',n_classes)
    # FIXME when torch will support where as np
    # assuming n_query, n_target constants
    n_query = target_cpu.eq(classes[0].item()).sum().item() - n_support
    #print('n_query:',n_query)
    support_idxs = list(map(supp_idxs, classes))

    prototypes = torch.stack([input_cpu[idx_list].mean(0) for idx_list in support_idxs])
    # FIXME when torch will support where as np
    query_idxs = torch.stack(list(map(lambda c: target_cpu.eq(c).nonzero()[n_support:], classes))).view(-1)

    query_samples = input.to(device)[query_idxs]
    dists = euclidean_dist(query_samples, prototypes)

    log_p_y = F.log_softmax(-dists, dim=1).view(n_classes, n_query, -1)

    target_inds = torch.arange(0, n_classes, device=device)
    target_inds = target_inds.view(n_classes, 1, 1)
    target_inds = target_inds.expand(n_classes, n_query, 1).long()
   # print('target_inds:',target_inds)

    loss_val = -log_p_y.gather(2, target_inds).squeeze().view(-1).mean()
    _, y_hat = log_p_y.max(2)
    #print('y_hat',y_hat)
    acc_val = y_hat.eq(target_inds.squeeze(2)).float().mean()
    target_inds = target_inds.squeeze(2)
    
    c_values = calculate_central_distance(prototypes, support_features, support_labels)
    #print(c_values)
    # Initialize class cluster loss
    cc_loss = 0.0
    
    # Compute Class Cluster Loss
    # Note: Assuming the prototypes calculation and euclidean_dist are defined as before
    # Loop over each class to calculate class cluster loss
    for i in range(4):#n_classes):
        # Assuming calculate_class_cluster_loss function calculates the loss for a single class
        cc_loss += class_cluster_loss(prototypes[i], query_features, query_labels, c_values[i],margin)
    
    # Normalize the class cluster loss by the number of classes
    #cc_loss /= n_classes
    #print('cc_loss',cc_loss,'\nloss_val',loss_val)
    

         # Combine the losses
    # Here, you might want to add a weighting factor to either loss depending on their importance
    total_loss = loss_val + cc_loss  # Adjust this combination based on your needs
 
    return total_loss,  acc_val


In [ ]:
class FewShotDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    

class PrototypicalBatchSampler(Sampler):
    def __init__(self, labels, classes_per_batch, support_per_class, queries_per_class):
        self.labels = labels
        self.classes_per_batch = classes_per_batch
        self.support_per_class = support_per_class
        self.queries_per_class = queries_per_class

        self.classes, self.counts = np.unique(self.labels, return_counts=True)
        self.indices = {c: np.where(self.labels == c)[0] for c in self.classes}

    def __iter__(self):
        for _ in range(len(self)):
            batch = []
            selected_classes = np.random.choice(self.classes, self.classes_per_batch, replace=False)
            
            for c in selected_classes:
                indices = np.random.choice(self.indices[c], self.support_per_class + self.queries_per_class, replace=False)
                batch.extend(indices)

            yield batch

    def __len__(self):
        return len(self.labels) // (self.classes_per_batch * (self.support_per_class + self.queries_per_class))


    #Test with other network design


class PrototypicalNetwork(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(PrototypicalNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )

    def forward(self, x):
        # Flatten the input tensor if it's not already a flat vector
        x = x.view(x.size(0), -1)  # Reshape input to [batch_size, input_size]
        return self.fc(x)


class PrototypicalNetwork2(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(PrototypicalNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )

    def forward(self, x):
        return self.fc(x)


    # coding=utf-8



class PrototypicalLoss(Module):
    '''
    Loss class deriving from Module for the prototypical loss function defined below
    '''
    def __init__(self, n_support):
        super(PrototypicalLoss, self).__init__()
        self.n_support = n_support

    def forward(self, input, target):
        return prototypical_loss(input, target)#, self.n_support)


def euclidean_dist(x, y):
    '''
    Compute euclidean distance between two tensors
    '''
    # x: N x D
    # y: M x D
    n = x.size(0)
    m = y.size(0)
    d = x.size(1)
    if d != y.size(1):
        raise Exception

    x = x.unsqueeze(1).expand(n, m, d)
    y = y.unsqueeze(0).expand(n, m, d)

    return torch.pow(x - y, 2).sum(2)



def prototypical_loss(input, target):#, n_support):
    '''
    Inspired by https://github.com/jakesnell/prototypical-networks/blob/master/protonets/models/few_shot.py

    Compute the barycentres by averaging the features of n_support
    samples for each class in target, computes then the distances from each
    samples' features to each one of the barycentres, computes the
    log_probability for each n_query samples for each one of the current
    classes, of appartaining to a class c, loss and accuracy are then computed
    and returned
    Args:
    - input: the model output for a batch of samples
    - target: ground truth for the above batch of samples
    - n_support: number of samples to keep in account when computing
      barycentres, for each one of the current classes
    '''
    target_cpu = target.to(device)
    input_cpu = input.to(device)

    def supp_idxs(c):
        # FIXME when torch will support where as np
        return target_cpu.eq(c).nonzero()[:n_support].squeeze(1)

    # FIXME when torch.unique will be available on cuda too
    classes = torch.unique(target_cpu)
    n_classes = len(classes)
    #print('n_classes:',n_classes)
    # FIXME when torch will support where as np
    # assuming n_query, n_target constants
    n_query = target_cpu.eq(classes[0].item()).sum().item() - n_support
    #print('n_query:',n_query)
    support_idxs = list(map(supp_idxs, classes))

    prototypes = torch.stack([input_cpu[idx_list].mean(0) for idx_list in support_idxs])
    # FIXME when torch will support where as np
    query_idxs = torch.stack(list(map(lambda c: target_cpu.eq(c).nonzero()[n_support:], classes))).view(-1)

    query_samples = input.to(device)[query_idxs]
    dists = euclidean_dist(query_samples, prototypes)

    log_p_y = F.log_softmax(-dists, dim=1).view(n_classes, n_query, -1)

    target_inds = torch.arange(0, n_classes, device=device)
    target_inds = target_inds.view(n_classes, 1, 1)
    target_inds = target_inds.expand(n_classes, n_query, 1).long()
   # print('target_inds:',target_inds)

    loss_val = -log_p_y.gather(2, target_inds).squeeze().view(-1).mean()
    _, y_hat = log_p_y.max(2)
    #print('y_hat',y_hat)
    acc_val = y_hat.eq(target_inds.squeeze(2)).float().mean()
    target_inds = target_inds.squeeze(2)
    
    
    return loss_val,  acc_val

def compute_prototypes(support_embeddings, support_labels, n_classes):
    """Compute class prototypes from support set embeddings."""
    prototypes = []
    for i in range(n_classes):
        # Extract embeddings belonging to class i
        class_embeddings = support_embeddings[support_labels == i]
        # Compute the mean of these embeddings
        prototype = class_embeddings.mean(0)
        prototypes.append(prototype)
    return torch.stack(prototypes)

def classify_embeddings(query_embeddings, prototypes):
    """Classify query embeddings based on the nearest class prototype."""
    # Calculate the distances from each query embedding to each prototype
    distances = euclidean_dist(query_embeddings, prototypes)

    # Find the index of the nearest prototype for each query embedding
    nearest_prototype_indices = distances.argmin(1)

    soft_assignments = F.softmax(-distances, dim=1) 
    
    return nearest_prototype_indices,soft_assignments

def get_support_set_embeddings(support_loader, proto_net):
    """Obtain embeddings and labels from the support set."""
    support_embeddings = []
    support_labels = []

    proto_net.eval()
    with torch.no_grad():
        for features, labels in support_loader:  # support_loader loads the support set
            
            features = features.to(device)
            optimizer.zero_grad()
            #dim_features, _ = dim_model(features)
        
            embeddings = proto_net(features)
            support_embeddings.append(embeddings.cpu().numpy())
            support_labels.append(labels.cpu().numpy())
        return np.concatenate(support_embeddings, axis=0), np.concatenate(support_labels, axis=0)


In [ ]:

dataset = ImageFolder(root='Alzheimer_s Dataset/train', transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
# assuming `your_dataset` is an instance of a class that extends torch.utils.data.Dataset

# Create your DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Calculate the number of instances in each class
class_counts = {}
for _, label in dataloader:
    label = label.item()
    if label in class_counts:
        class_counts[label] += 1
    else:
        class_counts[label] = 1

# Sort the counts for better visualization
sorted_counts = dict(sorted(class_counts.items()))

# Create a bar plot
plt.bar(sorted_counts.keys(), sorted_counts.values())
plt.xlabel('Class')
plt.ylabel('Number of instances')
plt.title('Distribution of instances per class')
plt.show()


In [ ]:

encoded_features = []
true_labels = []  # Assuming you have some labels
num_classes = 4

with torch.no_grad():
    for images, labels in data_loader:  # DataLoader for your labeled subset
        images = images.to(device)
        
        encoded_features.extend(images.cpu().numpy())
        true_labels.extend(labels.numpy())


X_train, X_test, y_train, y_test = train_test_split(encoded_features, true_labels, test_size=0.5, random_state=42)

In [ ]:
# Assuming 'data' and 'labels' are your datasets
train_dataset = FewShotDataset(X_train, y_train)
test_dataset = FewShotDataset(X_test, y_test)

train_sampler = PrototypicalBatchSampler(y_train, classes_per_batch=4, support_per_class=10, queries_per_class=5)
test_sampler = PrototypicalBatchSampler(y_test, classes_per_batch=4, support_per_class=10, queries_per_class=5)
 
train_loader = DataLoader(train_dataset, batch_sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_sampler=test_sampler)

In [ ]:
# Adjust the input_size to match the size of your extracted features
input_size =  224 * 224 * 3 #input_size = 64  # Replace with the actual size of your features
hidden_size = 128 #26  # 256 This can be adjusted as per your model's requirement


In [ ]:
proto_net = ResnetPrototypicalNetwork(input_size, hidden_size).to(device)
# Prototypical Loss
n_support = 10  # Adjust as needed
proto_loss_fn = PrototypicalLoss(n_support).to(device)
optimizer = torch.optim.Adam(proto_net.parameters(), lr=1e-4)

In [ ]:
epoch_losses = []
epoch_accuracies = []
avg_losses_for_plot = []  # To store average losses for plotting
avg_accuracies_for_plot = []  # To store average accuracies for plotting

# Training loop  hidden_size = 26 
for epoch in range(100):
    total_loss = 0.0
    total_accuracy = 0.0
    total_batches = 0

    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        optimizer.zero_grad()

        output = proto_net(features)
        loss, acc = proto_loss_fn(output, labels)

        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy
        total_loss += loss.item()
        total_accuracy += acc.item()
        total_batches += 1

    # Calculate average loss and accuracy for the epoch
    avg_loss = total_loss / total_batches
    avg_accuracy = total_accuracy / total_batches
    
    # Store metrics
    epoch_losses.append(avg_loss)
    epoch_accuracies.append(avg_accuracy)
    
    # Store metrics for plotting
    avg_losses_for_plot.append(avg_loss)
    avg_accuracies_for_plot.append(avg_accuracy)
    
    # Log the training progress
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}, Average Accuracy: {avg_accuracy:.4f}')


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have a DataLoader named support_loader for the support set
support_embeddings, support_labels = get_support_set_embeddings(test_loader, proto_net)
# Convert to PyTorch tensors
support_embeddings = torch.tensor(support_embeddings).to(device)
support_labels = torch.tensor(support_labels).to(device)

n_classes = 4
# Compute the prototypes
prototypes = compute_prototypes(support_embeddings, support_labels, n_classes)
# In your evaluation loop
with torch.no_grad():
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        query_embeddings = proto_net(features)
        predicted_labels = classify_embeddings(query_embeddings, prototypes)
proto_net.eval()  # Assuming 'proto_net' is your Prototypical Network model


from sklearn.metrics import accuracy_score

all_preds = []
all_labels = []

with torch.no_grad():
    for features, labels in test_loader:  # Replace eval_loader with your DataLoader for evaluation
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = proto_net(features)
        preds, soft_assignments = classify_embeddings(embeddings,prototypes)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')
print(classification_report(all_labels, all_preds, digits=4))



In [ ]:
# Save the model architecture and trained parameters
torch.save(proto_net.state_dict(), "ResNet_protonet.pth")
# Load the trained parameters
#model.load_state_dict(torch.load("trained_protonet.pth"))


In [ ]:
resnet = proto_net
# Assuming proto_net is your Prototypical Network
resnet.eval()  # Set the model to evaluation mode
 
embeddings = []
labels = []

with torch.no_grad():
    for features, batch_labels in test_loader:  # data_loader is your DataLoader
        features = features.to(device)
        #dim_features, _ = dim_model(features) 
        batch_embeddings = resnet(features)  # Get the embeddings
        embeddings.append(batch_embeddings.cpu().numpy())
        labels.append(batch_labels.cpu().numpy())

# Convert lists to numpy arrays
embeddings = np.concatenate(embeddings, axis=0)
labels = np.concatenate(labels, axis=0)
 



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.1, random_state=42)


In [ ]:
class_names=['MD','MOD','ND','VMD']

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.1, random_state=10)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.manifold import TSNE
 # Apply t-SNE transformation
tsne = TSNE(n_components=2, random_state=0)
embeddings_2d = tsne.fit_transform(X_test)
import seaborn as sns
palette = sns.color_palette(['blue', 'red', 'green', 'orange'])
# Define the marker styles if desired
markers = ['o']
# Plotting
plt.figure(figsize=(5, 4))
# Use seaborn to get a nicer plot
# Scatter plot for each class using the color palette
for i, color in zip(range(len(class_names)), palette):
    indices = y_test == i
    sns.scatterplot(x=embeddings_2d[indices , 0], y=embeddings_2d[indices , 1], 
                    label=f'{class_names[i]}', color=color, s=100, 
                    alpha=0.7, edgecolor='k', linewidth=0.5, marker=markers[i % len(markers)])

# Improving the legend and placing it outside the plot
#plt.legend(title='Classes', loc='center left', bbox_to_anchor=(1, 0.5), fontsize='large', title_fontsize='20')

# Adding title and labels with larger font sizes
plt.title('Embeddings of ProtoNet1 (ResNet18)', fontsize=14)
plt.xlabel('t-SNE x', fontsize=9)
plt.ylabel('t-SNE y', fontsize=9)

# Adding grid with dashed lines
plt.grid(True, linestyle='--')

# Tight layout often provides a better subplot arrangement
plt.tight_layout()
plt.savefig('emb_resnet1.png', dpi=300)

# Show the plot
plt.show()

## MobileNetPrototypicalNetwork

In [ ]:
import torch.nn as nn
import torchvision.models as models

class MobileNetPrototypicalNetwork(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MobileNetPrototypicalNetwork, self).__init__()
        # Load a pre-trained MobileNet model
        self.mobilenet = models.mobilenet_v2(pretrained=True)
        
        # Remove the classifier to use as a feature extractor
        # For MobileNetV2, features before the classifier are stored in 'features'
        self.features = self.mobilenet.features
        
        # MobileNetV2 outputs features of size 1280 for each image
        # Adjust the input size of the linear layer accordingly
        self.additional_layers = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),  # Global Average Pooling to reduce spatial dimensions
            nn.Flatten(),
            nn.Linear(1280, 4)  # Adjust the number here to match MobileNetV2's output
        )

    def forward(self, x):
        # Extract features using the MobileNet backbone
        x = self.features(x)
        
        # Pass through additional layers
        embedding = self.additional_layers(x)
        return embedding


In [ ]:
MobileNet_proto_net = MobileNetPrototypicalNetwork(input_size, hidden_size).to(device)
# Prototypical Loss
n_support = 10  # Adjust as needed
proto_loss_fn = PrototypicalLoss(n_support).to(device)
optimizer = torch.optim.Adam(MobileNet_proto_net.parameters(), lr=1e-4)
epoch_losses = []
epoch_accuracies = [] 
mobilenet_avg_losses_for_plot = []  # To store average losses for plotting
mobilenet_avg_accuracies_for_plot = []  # To store average accuracies for plotting

# Training loop  hidden_size = 26 
for epoch in range(100):
    total_loss = 0.0
    total_accuracy = 0.0
    total_batches = 0

    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        #optimizer.zero_grad()
        #output = proto_net(features)
        #loss, acc, predictions, true_labels = proto_loss_fn(output, labels)
        optimizer.zero_grad()

        # Extract features from the pre-trained DIM model
        #with torch.no_grad():
        #/    dim_features, _ = dim_model(features)

        # Now, use dim_features as the input to your ProtoNet model
        output = MobileNet_proto_net(features)
        loss, acc = proto_loss_fn(output, labels)


        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy
        total_loss += loss.item()
        total_accuracy += acc.item()
        total_batches += 1

    # Calculate average loss and accuracy for the epoch
    avg_loss = total_loss / total_batches
    avg_accuracy = total_accuracy / total_batches
    
    # Store metrics
    epoch_losses.append(avg_loss)
    epoch_accuracies.append(avg_accuracy)
    
    mobilenet_avg_losses_for_plot.append(avg_loss)
    mobilenet_avg_accuracies_for_plot.append(avg_accuracy)
    
    # Log the training progress
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}, Average Accuracy: {avg_accuracy:.4f}')


In [ ]:
# Assuming you have a DataLoader named support_loader for the support set
support_embeddings, support_labels = get_support_set_embeddings(test_loader, MobileNet_proto_net)
# Convert to PyTorch tensors
support_embeddings = torch.tensor(support_embeddings).to(device)
support_labels = torch.tensor(support_labels).to(device)

n_classes = 4
# Compute the prototypes
prototypes = compute_prototypes(support_embeddings, support_labels, n_classes)
# In your evaluation loop
with torch.no_grad():
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        query_embeddings = MobileNet_proto_net(features)
        predicted_labels = classify_embeddings(query_embeddings, prototypes)
MobileNet_proto_net.eval()  # Assuming 'proto_net' is your Prototypical Network model

all_preds = []
all_labels = []

with torch.no_grad():
    for features, labels in test_loader:  # Replace eval_loader with your DataLoader for evaluation
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = MobileNet_proto_net(features)
        preds, soft_assignments = classify_embeddings(embeddings,prototypes)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')
print(classification_report(all_labels, all_preds, digits=4))



In [ ]:
# Save the model architecture and trained parameters
torch.save(MobileNet_proto_net.state_dict(), "MobileNet_protonet.pth")

In [ ]:
MobileNet = MobileNet_proto_net
# Assuming you have embeddings_2d and y_test from your t-SNE and actual class data
# Here, we're going to use class_names for the legend instead of 'Class {i}'
MobileNet.eval()  # Set the model to evaluation mode

embeddings = []
labels = []

with torch.no_grad():
    for features, batch_labels in test_loader:  # data_loader is your DataLoader
        features = features.to(device)
        #dim_features, _ = dim_model(features) 
        batch_embeddings = MobileNet(features)  # Get the embeddings
        embeddings.append(batch_embeddings.cpu().numpy())
        labels.append(batch_labels.cpu().numpy())

# Convert lists to numpy arrays
embeddings = np.concatenate(embeddings, axis=0)
labels = np.concatenate(labels, axis=0)
 
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.1, random_state=42)

# Apply t-SNE transformation
tsne = TSNE(n_components=2, random_state=0)
embeddings_2d = tsne.fit_transform(X_test)

palette = sns.color_palette(['blue', 'red', 'green', 'orange'])
markers = ['o','o','o','o']  # Different markers for each class

# Plotting
plt.figure(figsize=(5, 4))

# Scatter plot for each class using the color palette and actual class names for the legend
for i, color in zip(range(len(class_names)), palette):
    indices = y_test == i
    sns.scatterplot(
        x=embeddings_2d[indices, 0], y=embeddings_2d[indices, 1], 
        label=class_names[i], color=color, s=100, 
        alpha=0.7, edgecolor='k', linewidth=0.5, marker=markers[i]
    )

# Improving the legend and placing it outside the plot
plt.legend(title='Classes',   bbox_to_anchor=(1, 0.5), fontsize='small', title_fontsize='9')

# Adding title and labels with larger font sizes
plt.title('Embeddings of ProtoNet3 (MobileNet)', fontsize=14)
plt.xlabel('t-SNE x', fontsize=9)
plt.ylabel('t-SNE y', fontsize=9)

# Adding grid with dashed lines
plt.grid(True, linestyle='--')

# Tight layout often provides a better subplot arrangement
plt.tight_layout()

# Save the figure
plt.savefig('emb_mobilenet.png', dpi=300)

# Show the plot
plt.show()
 

## VGGPrototypicalNetwork

In [ ]:
class VGGPrototypicalNetwork(nn.Module):
    def __init__(self, hidden_size):
        super(VGGPrototypicalNetwork, self).__init__()
        # Load a pre-trained VGG model
        self.vgg = models.vgg16(pretrained=True)
        
        # Use VGG as a feature extractor
        self.features = self.vgg.features
        self.avgpool = self.vgg.avgpool

        # Flatten the output and pass it through additional layers for feature embedding
        self.embedding = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, hidden_size),  # Embedding layer
            nn.ReLU(True)
            # Removed the final classification layer to focus on embeddings
        )

    def forward(self, x):
        # Extract features using the VGG backbone
        x = self.features(x)
        x = self.avgpool(x)

        # Generate embeddings suitable for prototypical network
        embedding = self.embedding(x)
        return embedding


In [ ]:
proto_net = VGGPrototypicalNetwork(128).to(device)
# Prototypical Loss
n_support = 10  # Adjust as needed
proto_loss_fn = PrototypicalLoss(n_support).to(device)
optimizer = torch.optim.Adam(proto_net.parameters(), lr=1e-4)
epoch_losses = []
epoch_accuracies = [] 
vgg_avg_losses_for_plot = []
vgg_avg_accuracies_for_plot = []


# Training loop  hidden_size = 26 
for epoch in range(100):
    total_loss = 0.0
    total_accuracy = 0.0
    total_batches = 0

    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        #optimizer.zero_grad()
        #output = proto_net(features)
        #loss, acc, predictions, true_labels = proto_loss_fn(output, labels)
        optimizer.zero_grad()

        # Extract features from the pre-trained DIM model
        #with torch.no_grad():
        #/    dim_features, _ = dim_model(features)

        # Now, use dim_features as the input to your ProtoNet model
        output = proto_net(features)
        #print(output.shape)
        loss, acc = proto_loss_fn(output, labels)


        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy
        total_loss += loss.item()
        total_accuracy += acc.item()
        total_batches += 1

    # Calculate average loss and accuracy for the epoch
    avg_loss = total_loss / total_batches
    avg_accuracy = total_accuracy / total_batches
    
    # Store metrics
    epoch_losses.append(avg_loss)
    epoch_accuracies.append(avg_accuracy)
    vgg_avg_losses_for_plot.append(avg_loss)
    vgg_avg_accuracies_for_plot.append(avg_accuracy)
    
    # Log the training progress
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}, Average Accuracy: {avg_accuracy:.4f}')


In [ ]:
# Assuming you have a DataLoader named support_loader for the support set
support_embeddings, support_labels = get_support_set_embeddings(test_loader,proto_net)
# Convert to PyTorch tensors
support_embeddings = torch.tensor(support_embeddings).to(device)
support_labels = torch.tensor(support_labels).to(device)

n_classes = 4
# Compute the prototypes
prototypes = compute_prototypes(support_embeddings, support_labels, n_classes)
# In your evaluation loop
with torch.no_grad():
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        query_embeddings = proto_net(features)
        predicted_labels = classify_embeddings(query_embeddings, prototypes)
proto_net.eval()  # Assuming 'proto_net' is your Prototypical Network model

all_preds = []
all_labels = []

with torch.no_grad():
    for features, labels in test_loader:  # Replace eval_loader with your DataLoader for evaluation
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = proto_net(features)
        preds, soft_assignments = classify_embeddings(embeddings,prototypes)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')
print(classification_report(all_labels, all_preds, digits=4))



In [ ]:
# Save the model architecture and trained parameters
torch.save(proto_net.state_dict(), "VGG_protonet.pth")
# Load the trained parameters
#model.load_state_dict(torch.load("trained_protonet.pth"))

In [ ]:

vgg = proto_net

vgg.eval()  # Set the model to evaluation mode

embeddings = []
labels = []

with torch.no_grad():
    for features, batch_labels in test_loader:  # data_loader is your DataLoader
        features = features.to(device)
        #dim_features, _ = dim_model(features) 
        batch_embeddings = vgg(features)  # Get the embeddings
        embeddings.append(batch_embeddings.cpu().numpy())
        labels.append(batch_labels.cpu().numpy())

# Convert lists to numpy arrays
embeddings = np.concatenate(embeddings, axis=0)
labels = np.concatenate(labels, axis=0)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.1, random_state=42)
 
 # Apply t-SNE transformation
tsne = TSNE(n_components=2, random_state=0)
embeddings_2d = tsne.fit_transform(X_test)

import seaborn as sns
palette = sns.color_palette(['blue', 'red', 'green', 'orange'])
# Define the marker styles if desired
markers = ['o']
# Plotting
plt.figure(figsize=(5, 4))
# Use seaborn to get a nicer plot
# Scatter plot for each class using the color palette
for i, color in zip(range(len(class_names)), palette):
    indices = y_test == i
    sns.scatterplot(x=embeddings_2d[indices , 0], y=embeddings_2d[indices , 1], 
                    label=f'{class_names[i]}', color=color, s=100, 
                    alpha=0.7, edgecolor='k', linewidth=0.5, marker=markers[i % len(markers)])

# Adding title and labels with larger font sizes
plt.title('Embeddings of ProtoNet5 (VGG16)', fontsize=14)
plt.xlabel('t-SNE x', fontsize=9)
plt.ylabel('t-SNE y', fontsize=9)


# Adding grid with dashed lines
plt.grid(True, linestyle='--')

# Tight layout often provides a better subplot arrangement
plt.tight_layout()
plt.savefig('emb_vgg.png', dpi=300)


# Show the plot
plt.show()

## EfficientNetPrototypicalNetwork

In [ ]:
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
import torchvision.models as models

class EfficientNetPrototypicalNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, efficientnet_version='b0', weights_path=None):
        super(EfficientNetPrototypicalNetwork, self).__init__()
        # Load a pre-trained EfficientNet model
        self.efficientnet = EfficientNet.from_pretrained(f'efficientnet-{efficientnet_version}', weights_path=weights_path)
        
        # Remove the classifier to use as a feature extractor
        # For EfficientNet, features before the classifier are stored in 'extract_features'
        self.features = self.efficientnet.extract_features
        
        # EfficientNet outputs features of variable size depending on the architecture
        # Adjust the input size of the linear layer accordingly
        self.additional_layers = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),  # Global Average Pooling to reduce spatial dimensions
            nn.Flatten()
        )
        # The output size of EfficientNet varies depending on its architecture
        # You may need to manually specify the input size for the linear layer
        self.linear = nn.Linear(self.efficientnet._fc.in_features, hidden_size)

    def forward(self, x):
        # Extract features using the EfficientNet backbone
        x = self.features(x)
        
        # Pass through additional layers
        x = self.additional_layers(x)
        
        # Linear layer to obtain the final embedding
        embedding = self.linear(x)
        return embedding


In [ ]:
proto_net = EfficientNetPrototypicalNetwork(input_size,hidden_size).to(device)
# Prototypical Loss
n_support = 10  # Adjust as needed
proto_loss_fn = PrototypicalLoss(n_support).to(device)
optimizer = torch.optim.Adam(proto_net.parameters(), lr=1e-4)
epoch_losses = []
epoch_accuracies = [] 
eff_avg_losses_for_plot = []
eff_avg_accuracies_for_plot = []


# Training loop  hidden_size = 26 
for epoch in range(100):
    total_loss = 0.0
    total_accuracy = 0.0
    total_batches = 0

    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        output = proto_net(features)
        loss, acc = proto_loss_fn(output, labels)
        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy
        total_loss += loss.item()
        total_accuracy += acc.item()
        total_batches += 1

    # Calculate average loss and accuracy for the epoch
    avg_loss = total_loss / total_batches
    avg_accuracy = total_accuracy / total_batches
    
    # Store metrics
    epoch_losses.append(avg_loss)
    epoch_accuracies.append(avg_accuracy)
    eff_avg_losses_for_plot.append(avg_loss)
    eff_avg_accuracies_for_plot.append(avg_accuracy)

    
    # Log the training progress
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}, Average Accuracy: {avg_accuracy:.4f}')


In [ ]:
# Assuming you have a DataLoader named support_loader for the support set
support_embeddings, support_labels = get_support_set_embeddings(test_loader,proto_net)
# Convert to PyTorch tensors
support_embeddings = torch.tensor(support_embeddings).to(device)
support_labels = torch.tensor(support_labels).to(device)

n_classes = 4
# Compute the prototypes
prototypes = compute_prototypes(support_embeddings, support_labels, n_classes)
# In your evaluation loop
with torch.no_grad():
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        query_embeddings = proto_net(features)
        predicted_labels = classify_embeddings(query_embeddings, prototypes)
proto_net.eval()  # Assuming 'proto_net' is your Prototypical Network model

all_preds = []
all_labels = []

with torch.no_grad():
    for features, labels in test_loader:  # Replace eval_loader with your DataLoader for evaluation
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = proto_net(features)
        preds, soft_assignments = classify_embeddings(embeddings,prototypes)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')
print(classification_report(all_labels, all_preds, digits=4))



In [ ]:
# Save the model architecture and trained parameters
torch.save(proto_net.state_dict(), "EfficientNet_protonet.pth")

In [ ]:
 # Assuming proto_net is your Prototypical Network
#efficientnet
proto_net.eval()  # Set the model to evaluation mode

embeddings = []
labels = []

with torch.no_grad():
    for features, batch_labels in test_loader:  # data_loader is your DataLoader
        features = features.to(device)
        #dim_features, _ = dim_model(features) 
        batch_embeddings = proto_net(features)  # Get the embeddings
        embeddings.append(batch_embeddings.cpu().numpy())
        labels.append(batch_labels.cpu().numpy())

# Convert lists to numpy arrays
embeddings = np.concatenate(embeddings, axis=0)
labels = np.concatenate(labels, axis=0)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.1, random_state=42)
 
 # Apply t-SNE transformation
tsne = TSNE(n_components=2, random_state=0)
embeddings_2d = tsne.fit_transform(X_test)

import seaborn as sns
palette = sns.color_palette(['blue', 'red', 'green', 'orange'])
# Define the marker styles if desired
markers = ['o']
# Plotting
plt.figure(figsize=(5, 4))
# Use seaborn to get a nicer plot
# Scatter plot for each class using the color palette
for i, color in zip(range(len(class_names)), palette):
    indices = y_test == i
    sns.scatterplot(x=embeddings_2d[indices , 0], y=embeddings_2d[indices , 1], 
                    label=f'{class_names[i]}', color=color, s=100, 
                    alpha=0.7, edgecolor='k', linewidth=0.5, marker=markers[i % len(markers)])

# Adding title and labels with larger font sizes
plt.title('Embeddings of ProtoNet4 (EfficientNet)', fontsize=14)
plt.xlabel('t-SNE x', fontsize=9)
plt.ylabel('t-SNE y', fontsize=9)



# Adding grid with dashed lines
plt.grid(True, linestyle='--')

# Tight layout often provides a better subplot arrangement
plt.tight_layout()
plt.savefig('emb_efficientnet.png', dpi=300)


# Show the plot
plt.show()

## ResNet34

In [ ]:
import torch.nn as nn
import torchvision.models as models

class Resnet34PrototypicalNetwork(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Resnet34PrototypicalNetwork, self).__init__()
        # Load a pre-trained ResNet model
        self.resnet = models.resnet34(pretrained=True)
        
        # Remove the fully connected layer (classifier) to use as a feature extractor
        self.features = nn.Sequential(*list(self.resnet.children())[:-1])
        
        # Flatten the output to [batch_size, 512] before passing to the linear layer
        self.additional_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, hidden_size)  # Adjust the number here to match ResNet's output
        )
        

    def forward(self, x):
        # Extract features using the ResNet backbone
        x = self.features(x)
        
        # Pass through any additional layers
        embedding = self.additional_layers(x)
        return embedding


In [ ]:
# Adjust the input_size to match the size of your extracted features
input_size =  224 * 224 * 3 #input_size = 64  # Replace with the actual size of your features
hidden_size = 128 #26  # 256 This can be adjusted as per your model's requirement

proto_net = Resnet34PrototypicalNetwork(input_size, hidden_size).to(device)
# Prototypical Loss
n_support = 10  # Adjust as needed
proto_loss_fn = PrototypicalLoss(n_support).to(device)
optimizer = torch.optim.Adam(proto_net.parameters(), lr=1e-4)

In [ ]:

epoch_losses = []
epoch_accuracies = [] 
resnet34_avg_losses_for_plot = []
resnet34_avg_accuracies_for_plot = []

# Training loop  hidden_size = 26 
for epoch in range(100):
    total_loss = 0.0
    total_accuracy = 0.0
    total_batches = 0

    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        #optimizer.zero_grad()
        #output = proto_net(features)
        #loss, acc, predictions, true_labels = proto_loss_fn(output, labels)
        optimizer.zero_grad()

        # Extract features from the pre-trained DIM model
        #with torch.no_grad():
        #/    dim_features, _ = dim_model(features)

        # Now, use dim_features as the input to your ProtoNet model
        output = proto_net(features)
        loss, acc = proto_loss_fn(output, labels)


        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy
        total_loss += loss.item()
        total_accuracy += acc.item()
        total_batches += 1

    # Calculate average loss and accuracy for the epoch
    avg_loss = total_loss / total_batches
    avg_accuracy = total_accuracy / total_batches
    
    # Store metrics
    epoch_losses.append(avg_loss)
    epoch_accuracies.append(avg_accuracy)
    resnet34_avg_losses_for_plot.append(avg_loss)
    resnet34_avg_accuracies_for_plot.append(avg_accuracy)
    # Log the training progress
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}, Average Accuracy: {avg_accuracy:.4f}')


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have a DataLoader named support_loader for the support set
support_embeddings, support_labels = get_support_set_embeddings(test_loader, proto_net)
# Convert to PyTorch tensors
support_embeddings = torch.tensor(support_embeddings).to(device)
support_labels = torch.tensor(support_labels).to(device)

n_classes = 4
# Compute the prototypes
prototypes = compute_prototypes(support_embeddings, support_labels, n_classes)
# In your evaluation loop
with torch.no_grad():
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        query_embeddings = proto_net(features)
        predicted_labels = classify_embeddings(query_embeddings, prototypes)
proto_net.eval()  # Assuming 'proto_net' is your Prototypical Network model


from sklearn.metrics import accuracy_score

all_preds = []
all_labels = []

with torch.no_grad():
    for features, labels in test_loader:  # Replace eval_loader with your DataLoader for evaluation
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = proto_net(features)
        preds, soft_assignments = classify_embeddings(embeddings,prototypes)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')
print(classification_report(all_labels, all_preds, digits=4))
 

In [ ]:
# Save the model architecture and trained parameters
torch.save(proto_net.state_dict(), "ResNet34_protonet.pth")

In [ ]:
 # Assuming proto_net is your Prototypical Network
resnet34.eval()  # Set the model to evaluation mode

embeddings = []
labels = []

with torch.no_grad():
    for features, batch_labels in test_loader:  # data_loader is your DataLoader
        features = features.to(device)
        #dim_features, _ = dim_model(features) 
        batch_embeddings = resnet34(features)  # Get the embeddings
        embeddings.append(batch_embeddings.cpu().numpy())
        labels.append(batch_labels.cpu().numpy())

# Convert lists to numpy arrays
embeddings = np.concatenate(embeddings, axis=0)
labels = np.concatenate(labels, axis=0)

X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.1, random_state=42)

# Apply t-SNE transformation
tsne = TSNE(n_components=2, random_state=0)
embeddings_2d = tsne.fit_transform(X_test)

import seaborn as sns
palette = sns.color_palette(['blue', 'red', 'green', 'orange'])
# Define the marker styles if desired
markers = ['o']
# Plotting
plt.figure(figsize=(5, 4))
# Use seaborn to get a nicer plot
# Scatter plot for each class using the color palette
for i, color in zip(range(len(class_names)), palette):
    indices = y_test == i
    sns.scatterplot(x=embeddings_2d[indices , 0], y=embeddings_2d[indices , 1], 
                    label=f'{class_names[i]}', color=color, s=100, 
                    alpha=0.7, edgecolor='k', linewidth=0.5, marker=markers[i % len(markers)])

# Adding title and labels with larger font sizes
plt.title('Embeddings of ProtoNet2 (ResNet34)', fontsize=14)
plt.xlabel('t-SNE x', fontsize=9)
plt.ylabel('t-SNE y', fontsize=9)

# Adding grid with dashed lines
plt.grid(True, linestyle='--')

# Tight layout often provides a better subplot arrangement
plt.tight_layout()
plt.savefig('emb_renet34.png', dpi=300)


# Show the plot
plt.show()

## Load trained models

In [ ]:
#Load models

resnet = ResnetPrototypicalNetwork(input_size, hidden_size).to(device)

resnet.load_state_dict(torch.load("ResNet18_protonet_class.pth"))

resnet34 = Resnet34PrototypicalNetwork(input_size, hidden_size).to(device)

resnet34.load_state_dict(torch.load("ResNet34_protonet_class.pth"))

eff = EfficientNetPrototypicalNetwork(128,4).to(device)

eff.load_state_dict(torch.load("EfficientNet_protonet_class.pth"))


mobilenet = MobileNetPrototypicalNetwork(input_size, hidden_size).to(device)

mobilenet.load_state_dict(torch.load("MobileNet_protonet_class.pth"))  

vgg = VGGPrototypicalNetwork(128).to(device)

vgg.load_state_dict(torch.load("vgg_protonet_class.pth"))  



In [ ]:
# Set the models to evaluation mode
resnet.eval()
resnet34.eval()
eff.eval()
mobilenet.eval()
vgg.eval()
optimizer = torch.optim.Adam(resnet.parameters(), lr=1e-4)

true_labels = []
# Initialize lists to store predictions and probabilities for each model
all_resnet_preds = []
all_resnet_labels = []
all_resnet_probs = []

all_resnet34_preds = []
all_resnet34_labels = []
all_resnet34_probs = []

all_eff_preds = []
all_eff_labels = []
all_eff_probs = []

all_m_preds = []
all_m_labels = []
all_m_probs = []

all_vgg_preds = []
all_vgg_labels = []
all_vgg_probs = []


# Assuming you have a DataLoader named support_loader for the support set
resnet_support_embeddings, resnet_support_labels = get_support_set_embeddings(test_loader, resnet)
# Convert to PyTorch tensors
resnet_support_embeddings = torch.tensor(resnet_support_embeddings).to(device)
resnet_support_labels = torch.tensor(resnet_support_labels).to(device)
# Compute the prototypes
resnet_prototypes = compute_prototypes(resnet_support_embeddings, resnet_support_labels, n_classes)


# Assuming you have a DataLoader named support_loader for the support set
resnet34_support_embeddings, resnet34_support_labels = get_support_set_embeddings(test_loader, resnet34)
# Convert to PyTorch tensors
resnet34_support_embeddings = torch.tensor(resnet34_support_embeddings).to(device)
resnet34_support_labels = torch.tensor(resnet34_support_labels).to(device)
# Compute the prototypes
resnet34_prototypes = compute_prototypes(resnet34_support_embeddings, resnet34_support_labels, n_classes)


# Assuming you have a DataLoader named support_loader for the support set
eff_support_embeddings, eff_support_labels = get_support_set_embeddings(test_loader, eff)
# Convert to PyTorch tensors
eff_support_embeddings = torch.tensor(eff_support_embeddings).to(device)
eff_support_labels = torch.tensor(eff_support_labels).to(device)
# Compute the prototypes
eff_prototypes = compute_prototypes(eff_support_embeddings, eff_support_labels, n_classes)

# Assuming you have a DataLoader named support_loader for the support set
m_support_embeddings, m_support_labels = get_support_set_embeddings(test_loader, mobilenet)
# Convert to PyTorch tensors
m_support_embeddings = torch.tensor(m_support_embeddings).to(device)
m_support_labels = torch.tensor(m_support_labels).to(device)
# Compute the prototypes
m_prototypes = compute_prototypes(m_support_embeddings, m_support_labels, n_classes)

# Assuming you have a DataLoader named support_loader for the support set
vgg_support_embeddings, vgg_support_labels = get_support_set_embeddings(test_loader, vgg)
# Convert to PyTorch tensors
vgg_support_embeddings = torch.tensor(vgg_support_embeddings).to(device)
vgg_support_labels = torch.tensor(vgg_support_labels).to(device)
# Compute the prototypes
vgg_prototypes = compute_prototypes(vgg_support_embeddings, vgg_support_labels, n_classes)


# In your evaluation loop
with torch.no_grad():
    
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        resnet_query_embeddings = resnet(features)
        preds, soft_assignments = classify_embeddings(resnet_query_embeddings, resnet_prototypes)
        all_resnet_preds.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
        all_resnet_labels.extend(labels.cpu().numpy())
        all_resnet_probs.extend(soft_assignments.cpu().numpy())  # Collect soft assignment probabilities

        
        resnet34_query_embeddings = resnet34(features)
        preds, soft_assignments = classify_embeddings(resnet34_query_embeddings, resnet34_prototypes)
        all_resnet34_preds.extend(preds.cpu().numpy())
        all_resnet34_labels.extend(labels.cpu().numpy())
        all_resnet34_probs.extend(soft_assignments.cpu().numpy())  # Collect soft assignment probabilities

        eff_query_embeddings = eff(features)
        preds, soft_assignments = classify_embeddings(eff_query_embeddings, eff_prototypes)
        all_eff_preds.extend(preds.cpu().numpy())
        all_eff_labels.extend(labels.cpu().numpy())
        all_eff_probs.extend(soft_assignments.cpu().numpy())  # Collect soft assignment probabilities
                        
        m_query_embeddings = mobilenet(features)
        preds, soft_assignments = classify_embeddings(m_query_embeddings, m_prototypes)
        all_m_preds.extend(preds.cpu().numpy())
        all_m_labels.extend(labels.cpu().numpy())
        all_m_probs.extend(soft_assignments.cpu().numpy())  # Collect soft assignment probabilities
        
        vgg_query_embeddings = vgg(features)
        preds, soft_assignments = classify_embeddings(vgg_query_embeddings, vgg_prototypes)
        all_vgg_preds.extend(preds.cpu().numpy())
        all_vgg_labels.extend(labels.cpu().numpy())
        all_vgg_probs.extend(soft_assignments.cpu().numpy())  # Collect soft assignment probabilities



In [ ]:
# Calculate accuracy
accuracy = accuracy_score(all_resnet34_labels, all_resnet34_preds)
print("----------------ResNet34--------------------\n",f'Accuracy: {accuracy}')
print(classification_report(all_resnet34_labels, all_resnet34_preds, digits=4))

accuracy = accuracy_score(all_resnet_labels, all_resnet_preds)
print("----------------ResNet18--------------------\n",f'Accuracy: {accuracy}')
print(classification_report(all_resnet_labels, all_resnet_preds, digits=4))

accuracy = accuracy_score(all_eff_labels, all_eff_preds)
print("----------------EfficientNEt--------------------\n",f'Accuracy: {accuracy}')
print(classification_report(all_eff_labels, all_eff_preds, digits=4))
                          
accuracy = accuracy_score(all_m_labels, all_m_preds)
print("----------------MobileNEt--------------------\n",f'Accuracy: {accuracy}')
print(classification_report(all_m_labels, all_m_preds, digits=4))

accuracy = accuracy_score(all_vgg_labels, all_vgg_preds)
print("----------------VGG16--------------------\n",f'Accuracy: {accuracy}')
print(classification_report(all_vgg_labels, all_vgg_preds, digits=4))



## Hard Voting 

In [ ]:
from scipy import stats
# Predict labels with models
labels = []
labels.append(all_resnet_preds)
labels.append(all_resnet34_preds)
labels.append(all_eff_preds)
labels.append(all_vgg_preds)
labels.append(all_m_preds)
labels=np.asarray(labels)
x=labels.T
labels = np.squeeze(x)
###Mode Pdrediction
mode_info = stats.mode(labels, axis = 1)
f = np.squeeze(mode_info[0]) 

accuracy=accuracy_score(true_labels, f)
print("Hard Voting\n Accuracy: ", accuracy)
print(classification_report(true_labels, f, digits=4))


In [ ]:
from scipy import stats
# Predict labels with models
labels = []
labels.append(all_resnet_preds)
labels.append(all_resnet34_preds)
labels.append(all_eff_preds)
#labels.append(all_m_preds)
labels=np.asarray(labels)
x=labels.T
labels = np.squeeze(x)
###Mode Pdrediction
mode_info = stats.mode(labels, axis = 1)
f = np.squeeze(mode_info[0]) 

accuracy=accuracy_score(true_labels, f)
print("Hard Voting\n Accuracy: ", accuracy)
print(classification_report(true_labels, f, digits=4))


In [ ]:

accuracy=accuracy_score(true_labels, f)
print("Hard Voting\n Accuracy: ", accuracy)
print(classification_report(true_labels, f, digits=4))


In [ ]:
# Assuming you have a PyTorch dataset object named 'dataset'
class_names = dataset.classes
class_names=['MD', 'MOD', 'ND', 'VMD']
class_names
# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, f)

# Normalize confusion matrix
conf_matrix_norm = normalize(conf_matrix, axis=1, norm='l1')

# Define class labels
classes = class_names

# Plot normalized confusion matrix with a different colormap
plt.figure(figsize=(5, 4))
sns.set(font_scale=1.2)  # Adjust font scale for better readability
sns.heatmap(conf_matrix_norm, annot=True, cmap='blues', fmt='.2f', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Normalized Confusion Matrix')
plt.savefig('CM96.png')
plt.show()


In [ ]:
# Assuming you have a PyTorch dataset object named 'dataset'
class_names = dataset.classes
class_names=['Mild', 'Moderate', 'Non', 'VeryMild']
class_names

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, f)

# Normalize confusion matrix
conf_matrix_norm = normalize(conf_matrix, axis=1, norm='l1')

# Define class labels
classes = class_names

# Plot normalized confusion matrix with a different colormap
plt.figure(figsize=(5, 4))
sns.set(font_scale=1.2)  # Adjust font scale for better readability
sns.heatmap(conf_matrix_norm, annot=True, cmap='blues', fmt='.2f', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Normalized Confusion Matrix')
plt.show()


## Soft Voting

In [ ]:
from scipy import stats

# Collect predicted probabilities from models
probabilities = []
probabilities.append(all_resnet_probs)
probabilities.append(all_resnet34_probs)
probabilities.append(all_m_probs)
probabilities.append(all_eff_probs)
probabilities.append(all_vgg_probs)

# Convert to numpy array
probabilities = np.asarray(probabilities)

# Transpose to have shape (samples, models, classes)
probabilities = probabilities.transpose(1, 2, 0)

# Calculate average probabilities across models
average_probabilities = np.mean(probabilities, axis=2)

# Get final predictions
soft_voting_predictions = np.argmax(average_probabilities, axis=1)


In [ ]:
accuracy=accuracy_score(true_labels, soft_voting_predictions)
print("Soft Voting\n Accuracy: ", accuracy)
print(classification_report(true_labels, soft_voting_predictions, digits=4))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, soft_voting_predictions)

# Normalize confusion matrix
conf_matrix_norm = normalize(conf_matrix, axis=1, norm='l1')

# Define class labels
classes = class_names

# Plot normalized confusion matrix with a different colormap
plt.figure(figsize=(5, 4))
sns.set(font_scale=1.2)  # Adjust font scale for better readability
sns.heatmap(conf_matrix_norm, annot=True, cmap='YlGnBu', fmt='.2f', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Normalized Confusion Matrix')
plt.show()


In [ ]:
 import matplotlib.pyplot as plt
import numpy as np

categories = ['MD', 'MOD', 'ND', 'VMD']
precision_proto = [98.23, 99.4, 97.29, 95.01]
recall_proto = [96.67, 99, 96.98, 96.67]
f1_proto = [97.44, 99.2, 97.14, 95.83]

# Dummy data for Proposed Model (SV) Classification Scores
precision_proposed = [100, 100, 99.84, 99.37]
recall_proposed = [100, 100, 99.84, 98.84]
f1_proposed = [99.76, 100, 99.84, 99.60]

# Creating a figure to hold the subplots
plt.figure(figsize=(10, 3))  # Adjust the figure size as needed

# Precision subplot
plt.subplot(1, 3, 1)  # 1 row, 3 columns, 1st subplot
plt.plot(categories, precision_proto, marker='o', label='ProtoNet4', linestyle='dashed', color='blue')
plt.plot(categories, precision_proposed, marker='o', label='Proposed Model', color='blue')
plt.ylabel('Score (%)')
plt.ylim(90, 103)
plt.title('Precision',fontsize=18)
plt.legend()
plt.grid(True)

# Recall subplot
plt.subplot(1, 3, 2)  # 1 row, 3 columns, 2nd subplot
plt.plot(categories, recall_proto, marker='o', label='ProtoNet4 ', linestyle='dashed', color='red')
plt.plot(categories, recall_proposed, marker='o', label='Proposed Model', color='red')
plt.ylabel('Score (%)')
plt.title('Recall',fontsize=18)
plt.ylim(90, 103)
plt.legend()
plt.grid(True)

# F1-Score subplot
plt.subplot(1, 3, 3)  # 1 row, 3 columns, 3rd subplot
plt.plot(categories, f1_proto, marker='o', label='ProtoNet4 ', linestyle='dashed', color='green')
plt.plot(categories, f1_proposed, marker='o', label='Proposed Model', color='green')
plt.ylabel('Score (%)')
plt.title('F1-Score',fontsize=18)
plt.ylim(90, 103)
plt.legend()
plt.grid(True)

# Modifying the spines for all plots
for i in range(1, 4):
    ax = plt.subplot(1, 3, i)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

plt.tight_layout()  # Adjusts subplot params so that subplots are nicely fit in the figure area.
plt.savefig('comp_metric_wise_horizontal.png', dpi=300)
plt.show()


## Protonet (CNN)

In [ ]:
#Test with other network design

import torch
import torch.nn as nn

class PrototypicalNetwork(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(PrototypicalNetwork, self).__init__()
        self.flatten = nn.Flatten()  # Add a flattening layer to flatten the input tensor
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )

    def forward(self, x):
        x = self.flatten(x)  # Flatten the input tensor
        return self.fc(x)


    # coding=utf-8
import torch
from torch.nn import functional as F
from torch.nn.modules import Module


class PrototypicalLoss(Module):
    '''
    Loss class deriving from Module for the prototypical loss function defined below
    '''
    def __init__(self, n_support):
        super(PrototypicalLoss, self).__init__()
        self.n_support = n_support

    def forward(self, input, target):
        return prototypical_loss(input, target)#, self.n_support)


def euclidean_dist(x, y):
    '''
    Compute euclidean distance between two tensors
    '''
    # x: N x D
    # y: M x D
    n = x.size(0)
    m = y.size(0)
    d = x.size(1)
    if d != y.size(1):
        raise Exception

    x = x.unsqueeze(1).expand(n, m, d)
    y = y.unsqueeze(0).expand(n, m, d)

    return torch.pow(x - y, 2).sum(2)


def prototypical_loss(input, target):#, n_support):
    '''
    Inspired by https://github.com/jakesnell/prototypical-networks/blob/master/protonets/models/few_shot.py

    Compute the barycentres by averaging the features of n_support
    samples for each class in target, computes then the distances from each
    samples' features to each one of the barycentres, computes the
    log_probability for each n_query samples for each one of the current
    classes, of appartaining to a class c, loss and accuracy are then computed
    and returned
    Args:
    - input: the model output for a batch of samples
    - target: ground truth for the above batch of samples
    - n_support: number of samples to keep in account when computing
      barycentres, for each one of the current classes
    '''
    target_cpu = target.to(device)
    input_cpu = input.to(device)

    def supp_idxs(c):
        # FIXME when torch will support where as np
        return target_cpu.eq(c).nonzero()[:n_support].squeeze(1)

    # FIXME when torch.unique will be available on cuda too
    classes = torch.unique(target_cpu)
    n_classes = len(classes)
    #print('n_classes:',n_classes)
    # FIXME when torch will support where as np
    # assuming n_query, n_target constants
    n_query = target_cpu.eq(classes[0].item()).sum().item() - n_support
    #print('n_query:',n_query)
    support_idxs = list(map(supp_idxs, classes))

    prototypes = torch.stack([input_cpu[idx_list].mean(0) for idx_list in support_idxs])
    # FIXME when torch will support where as np
    query_idxs = torch.stack(list(map(lambda c: target_cpu.eq(c).nonzero()[n_support:], classes))).view(-1)

    query_samples = input.to(device)[query_idxs]
    dists = euclidean_dist(query_samples, prototypes)

    log_p_y = F.log_softmax(-dists, dim=1).view(n_classes, n_query, -1)

    target_inds = torch.arange(0, n_classes, device=device)
    target_inds = target_inds.view(n_classes, 1, 1)
    target_inds = target_inds.expand(n_classes, n_query, 1).long()
   # print('target_inds:',target_inds)

    loss_val = -log_p_y.gather(2, target_inds).squeeze().view(-1).mean()
    _, y_hat = log_p_y.max(2)
    #print('y_hat',y_hat)
    acc_val = y_hat.eq(target_inds.squeeze(2)).float().mean()
    target_inds = target_inds.squeeze(2)
    return loss_val,  acc_val, y_hat, target_inds

In [ ]:
# Adjust the input_size to match the size of your extracted features
input_size = 224*224*3  # Replace with the actual size of your features
hidden_size = 128 #26  # 256 This can be adjusted as per your model's requirement

proto_net = PrototypicalNetwork(input_size, hidden_size).to(device)
# Prototypical Loss
n_support = 10  # Adjust as needed
proto_loss_fn = PrototypicalLoss(n_support).to(device)
optimizer = torch.optim.Adam(proto_net.parameters(), lr=1e-4)

In [ ]:
epoch_losses = []
epoch_accuracies = [] 

# Training loop  hidden_size = 26 
for epoch in range(100):
    total_loss = 0.0
    total_accuracy = 0.0
    total_batches = 0

    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        optimizer.zero_grad()
        output = proto_net(features)
        loss, acc, predictions, true_labels = proto_loss_fn(output, labels)
        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy
        total_loss += loss.item()
        total_accuracy += acc.item()
        total_batches += 1

    # Calculate average loss and accuracy for the epoch
    avg_loss = total_loss / total_batches
    avg_accuracy = total_accuracy / total_batches
    
    # Store metrics
    epoch_losses.append(avg_loss)
    epoch_accuracies.append(avg_accuracy)
    
    # Log the training progress
    print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}, Average Accuracy: {avg_accuracy:.4f}')


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have a DataLoader named support_loader for the support set
support_embeddings, support_labels = get_support_set_embeddings(test_loader, proto_net)
# Convert to PyTorch tensors
support_embeddings = torch.tensor(support_embeddings).to(device)
support_labels = torch.tensor(support_labels).to(device)

n_classes = 4
# Compute the prototypes
prototypes = compute_prototypes(support_embeddings, support_labels, n_classes)
# In your evaluation loop
with torch.no_grad():
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        query_embeddings = proto_net(features)
        predicted_labels = classify_embeddings(query_embeddings, prototypes)
proto_net.eval()  # Assuming 'proto_net' is your Prototypical Network model


from sklearn.metrics import accuracy_score

all_preds = []
all_labels = []

with torch.no_grad():
    for features, labels in test_loader:  # Replace eval_loader with your DataLoader for evaluation
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = proto_net(features)
        preds, soft_assignments = classify_embeddings(embeddings,prototypes)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')
print(classification_report(all_labels, all_preds, digits=4))

